# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.116813e+02     1.551620e+00
 * time: 0.7788770198822021
     1     1.081487e+01     9.126928e-01
 * time: 2.1688549518585205
     2    -1.172819e+01     9.878274e-01
 * time: 2.8449771404266357
     3    -3.387297e+01     8.382242e-01
 * time: 3.846935987472534
     4    -4.723665e+01     6.650109e-01
 * time: 4.844261169433594
     5    -5.687043e+01     2.810925e-01
 * time: 5.812992095947266
     6    -5.967900e+01     2.618117e-01
 * time: 6.501569032669067
     7    -6.087712e+01     1.054548e-01
 * time: 7.231588125228882
     8    -6.141503e+01     3.996204e-02
 * time: 7.945536136627197
     9    -6.168234e+01     3.221613e-02
 * time: 8.644531011581421
    10    -6.185873e+01     3.193582e-02
 * time: 9.311486959457397
    11    -6.200140e+01     2.088493e-02
 * time: 9.988492012023926
    12    -6.208674e+01     1.518422e-02
 * time: 10.659409046173096
    13    -6.212964e+01     1.156476e-02
 * time: 11.332324028015137
  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671063
    AtomicLocal         -18.8557620
    AtomicNonlocal      14.8522602
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485357 
    Xc                  -19.3336815

    total               -62.261666457739
